In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
adni_bids = '/scratch/yx2105/shared/MLH/data/bids_part2/'
adni_clinical_bids = '/scratch/yx2105/shared/MLH/data/clinical_bids_part2/'
subject_sessions_list_file =  'subject_sessions_list_2.tsv'


adni_bids = '/scratch/yx2105/shared/MLH/data/bids_part1/'
adni_clinical_bids = '/scratch/yx2105/shared/MLH/data/bids_part1/'
subject_sessions_list_file =  'subject_sessions_list_1.tsv'


# 1. Prepare clinical data

In [ ]:
# Examples of columns that can be used from ADNI BIDS:
participant_columns = ['alternative_id_1', 'participant_id',"sex", "education_level","marital_status", "apoe4", "apoe_gen1", "apoe_gen2","diagnosis_sc"]

session_columns = ["age",
                   # Cognitive measures
                   "MMSE", "cdr_sb", "cdr_global", "adas11", "adas13",
                   "adas_memory", "adas_language", "adas_concentration", "adas_praxis", "ravlt_immediate", "moca",
                   "TMT_A", "TMT_B", "dsst", "logmem_delay", "logmem_imm",
                   # RAVLT score
                   "neurobat_ravlt_forgetting", "neurobat_ravlt_learning", "neurobat_ravlt_perc_forgetting",
                   # everyday cognition test score
                   "ecogpt_ecogpttotal", "ecogsp_ecogpttotal",
                   # T1 measures
                   "adni_ventricles_vol", "adni_hippocampus_vol", "adni_brain_vol", "adni_entorhinal_vol",
                   "adni_fusiform_vol", "adni_midtemp_vol", "adni_icv",
                   # PET measures
                   "adni_fdg", "adni_pib", "adni_av45",
                   # CSF measures
                   "adni_abeta", "adni_tau", "adni_ptau"]

In [ ]:
participants_tsv = pd.read_csv(os.path.join(adni_bids, "participants.tsv"), sep='\t')
subj_sessions = pd.read_csv(os.path.join('/scratch/yx2105/shared/MLH/data/', subject_sessions_list_file) , sep='\t')


In [ ]:
participants_tsv.columns

In [ ]:
participant_series = {}
session_series = {}
for col in participant_columns:
    participant_series[col] = []
for col in session_columns:
    session_series[col] = []

In [ ]:
# We collect the specified columns data
for row in subj_sessions.iterrows():
    subj_sess = row[1]
    # From the participants.tsv file for each subject
    selected_participant = participants_tsv[(participants_tsv.participant_id == subj_sess.participant_id)].iloc[0]
    for col in participant_columns:
        participant_series[col].append(selected_participant[col])
        

    # From the different sessions.tsv files for each subject and session
    session_tsv = pd.read_csv(os.path.join(adni_clinical_bids, subj_sess.participant_id,
                                        subj_sess.participant_id + "_sessions.tsv"), sep='\t')
    selected_session = session_tsv[(session_tsv.session_id == subj_sess.session_id)].iloc[0]
    for col in session_columns:
        if col in selected_session:
            session_series[col].append(selected_session[col])
        else:
            session_series[col].append(np.nan)
      

In [ ]:
# We add collected information to subjects .tsv
for col in participant_columns:
    subj_sessions.loc[:, col] = pd.Series(participant_series[col], index=subj_sessions.index)

for col in session_columns:
    subj_sessions.loc[:, col] = pd.Series(session_series[col], index=subj_sessions.index)


In [ ]:
# We replace gender information that is text by numeric values
subj_sessions.loc[subj_sessions[subj_sessions.sex == 'F'].index, 'sex'] = 1
subj_sessions.loc[subj_sessions[subj_sessions.sex == 'M'].index, 'sex'] = 0


In [ ]:
subj_sessions.to_csv(os.path.join(adni_clinical_bids, 'all_clinical_data.tsv'), sep='\t', index=False)


## Load data and create labels

In [ ]:
subj_sessions_1 = pd.read_csv(os.path.join('/scratch/yx2105/shared/MLH/data/bids_part1', 'all_clinical_data.tsv'), sep='\t')
subj_sessions_1['data_dir'] = '/scratch/yx2105/shared/MLH/data/bids_part1'
subj_sessions_2 = pd.read_csv(os.path.join('/scratch/yx2105/shared/MLH/data/clinical_bids_part2/', 'all_clinical_data.tsv'), sep='\t')
subj_sessions_2['data_dir'] = '/scratch/yx2105/shared/MLH/data/bids_part2'
subj_sessions = pd.concat([subj_sessions_1, subj_sessions_2], ignore_index=True, sort=False)
subj_sessions = subj_sessions.sort_values(['participant_id','session_id']).reset_index(drop=True)



In [ ]:
subj_sessions#.describe()

In [ ]:
# get next session time and label

img_dirs = []
diagnosis_12months = []
for id, row in subj_sessions.iterrows():
    participant_id = row['participant_id']
    session_id = row['session_id']
    session_id_12 = 'ses-M{}'.format(int(session_id.split('M')[-1]) + 12)
    row_12 = subj_sessions[(subj_sessions['participant_id'] == participant_id) & (subj_sessions['session_id'] == session_id_12)]
    if len(row_12) == 0:
        diagnosis_12months.append('')
    else:
        diagnosis_12months.append(row_12.iloc[0]['diagnosis_sc'])

    file_name = '{}_{}_{}'.format(participant_id,session_id,'T1w.nii.gz')
    img_dir = os.path.join(row['data_dir'],participant_id,session_id,'anat',file_name)
    exist = os.path.exists(img_dir)
    if exist:
        img_dirs.append(img_dir)
    else:
        img_dirs.append('')

subj_sessions['diagnosis_12month'] = diagnosis_12months
subj_sessions['img_dir'] = img_dirs


In [ ]:
subj_sessions_12month = subj_sessions[(subj_sessions['diagnosis_12month'] != '') &(subj_sessions['diagnosis_12month'] != 'SMC')]
subj_sessions_12month = subj_sessions_12month[(subj_sessions['img_dir'] != '')]


In [ ]:
subj_sessions_12month

In [ ]:
model_12month = subj_sessions_12month[~subj_sessions.sex.isnull() &
                        ~subj_sessions.education_level.isnull() &
                        ~subj_sessions.apoe4.isnull() &
                        ~subj_sessions.MMSE.isnull() &
                        ~subj_sessions.cdr_sb.isnull() &
                        ~subj_sessions.adas_memory.isnull() &
                        ~subj_sessions.adas_language.isnull() &
                        ~subj_sessions.adas_concentration.isnull() &
                        ~subj_sessions.adas_praxis.isnull() &
                        ~subj_sessions.ravlt_immediate.isnull()].reset_index()

model_all = subj_sessions[~subj_sessions.sex.isnull() &
                        ~subj_sessions.education_level.isnull() &
                        ~subj_sessions.apoe4.isnull() &
                        ~subj_sessions.MMSE.isnull() &
                        ~subj_sessions.cdr_sb.isnull() &
                        ~subj_sessions.adas_memory.isnull() &
                        ~subj_sessions.adas_language.isnull() &
                        ~subj_sessions.adas_concentration.isnull() &
                        ~subj_sessions.adas_praxis.isnull() &
                        ~subj_sessions.ravlt_immediate.isnull()]

## Data cleaning

In [ ]:
all_df = subj_sessions_12month

# remove columns that has more than 70% missing values
new_cols = []
for col in all_df.columns:
    if (all_df.isnull().sum()/len(all_df) < 0.7)[col]:
        new_cols.append(col)
    
all_df = all_df[new_cols]

In [ ]:
all_df.columns

In [ ]:
all_missing = all_df.isnull().sum()/len(all_df)

In [ ]:
cat_col = []
for col in all_df:
    if len(all_df[col].unique()) < 25:
        print(col,len(all_df[col].unique()),all_df[col].unique())
        cat_col.append(col)

In [ ]:
all_missing[cat_col]

In [ ]:
clean_df = all_df

# impute for categorical variables
clean_df.loc[:,'apoe4'] = clean_df['apoe4'].fillna(clean_df['apoe4'].mode().iloc[0])
clean_df.loc[:,'cdr_global'] = clean_df['cdr_global'].fillna(clean_df['cdr_global'].mode().iloc[0])

clean_df.loc[:,['apoe_gen1','apoe_gen2','adas_concentration']] = clean_df[['apoe_gen1','apoe_gen2','adas_concentration']].fillna(999)

In [ ]:
# impute for numerical variable
num_cols_small = ['age','MMSE', 'cdr_sb','adas11','adas13', 'ravlt_immediate']
clean_df[num_cols_small] = clean_df[num_cols_small].fillna(clean_df[num_cols_small].median())


In [ ]:
# get df with missing values
missing_df = clean_df

In [ ]:
# reduce the size of the dataset
# clean_df = clean_df[~clean_df.logmem_delay.isnull()]

In [ ]:
# drop numerical cols with too large missing values
clean_df = clean_df.drop(['adas_memory','adas_language', 'adas_praxis', 'dsst','adni_fdg'], axis=1)


In [ ]:
# deal with relatively large missing values with mean
num_cols_large = ['moca','logmem_delay', 'logmem_imm','adni_ventricles_vol','adni_hippocampus_vol', 'adni_brain_vol','adni_entorhinal_vol','adni_fusiform_vol',
'adni_midtemp_vol', 'adni_icv']

clean_df[num_cols_large] = clean_df[num_cols_large].fillna(clean_df[num_cols_large].mean())


In [ ]:
#clean_df = clean_df.drop(num_cols_large,axis = 1)
clean_df.isnull().sum()

## Data processing

In [ ]:
processed_df = clean_df
cat_col = [
 'marital_status',
 'apoe4',
 'apoe_gen1',
 'apoe_gen2',
 'cdr_global',
 'adas_concentration']

dummy_df = pd.get_dummies(processed_df[cat_col].astype('category'))

processed_df = processed_df.drop(cat_col,axis=1)
processed_df = pd.concat([processed_df,dummy_df],axis=1,join = 'inner').reset_index(drop=True)

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

# split data into train and test sets based on the "id" column
X = processed_df.drop("diagnosis_12month", axis=1)
y = processed_df["diagnosis_12month"]
gss = GroupShuffleSplit(n_splits=100, test_size=0.2)

for train_index, test_index in gss.split(X, y, groups=processed_df["participant_id"]):
    X_train_all, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train_all, y_test = y.iloc[train_index], y.iloc[test_index]

X_train_all = X_train_all.reset_index(drop=True)
y_train_all = y_train_all.reset_index(drop=True)
print(len(X_train_all))

gss2 = GroupShuffleSplit(n_splits=100, test_size=0.2)
for train_index, val_index in gss2.split(X_train_all, y_train_all, groups=X_train_all["participant_id"]):
    X_train, X_val = X_train_all.iloc[train_index], X_train_all.iloc[val_index]
    y_train, y_val = y_train_all.iloc[train_index], y_train_all.iloc[val_index]


In [ ]:
train = pd.concat([X_train,y_train],axis=1).reset_index(drop=True)
val = pd.concat([X_val,y_val],axis=1).reset_index(drop=True)
test = pd.concat([X_test,y_test],axis=1).reset_index(drop=True)

In [ ]:
remove_list = ['participant_id','session_id','alternative_id_1','diagnosis_sc','diagnosis_12month', 'data_dir','img_dir']
tab_columns = [i for i in train.columns if i not in remove_list]

X_train = train.loc[:,tab_columns]
X_val = val.loc[:,tab_columns]
X_test = test.loc[:,tab_columns]

# create a StandardScaler object
scaler = StandardScaler()

# fit the scaler to the training data
scaler.fit(X_train)

# transform the training and test sets using the scaler
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

train.loc[:,tab_columns] = X_train_scaled
val.loc[:,tab_columns] = X_val_scaled
test.loc[:,tab_columns] = X_test_scaled

# train.to_csv('/scratch/yx2105/shared/MLH/data/train_large.csv')
# val.to_csv('/scratch/yx2105/shared/MLH/data/val_large.csv')
# test.to_csv('/scratch/yx2105/shared/MLH/data/test_large.csv')

## Load processed data list

In [ ]:
train = pd.read_csv('/scratch/yx2105/shared/MLH/data/train.csv',header=0,index_col=0)
val = pd.read_csv('/scratch/yx2105/shared/MLH/data/val.csv',header=0,index_col=0)
test = pd.read_csv('/scratch/yx2105/shared/MLH/data/test.csv',header=0,index_col=0)


In [ ]:
len(test)

In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt 
%matplotlib inline

train = pd.read_csv('/scratch/yx2105/shared/MLH/data/train.csv',header=0,index_col=0)
val = pd.read_csv('/scratch/yx2105/shared/MLH/data/val.csv',header=0,index_col=0)
test = pd.read_csv('/scratch/yx2105/shared/MLH/data/test.csv',header=0,index_col=0)

mapping = {
    'CN': 0,
    'AD': 1,
    'LMCI':2,
    'EMCI':2,
    'MCI':2,
    'SMC':2 
}

train['diagnosis_12month'] = train['diagnosis_12month'].astype("category").map(mapping)
val['diagnosis_12month'] = val['diagnosis_12month'].astype("category").map(mapping)
test['diagnosis_12month'] = test['diagnosis_12month'].astype("category").map(mapping)


remove_list = ['participant_id','session_id','alternative_id_1','diagnosis_sc','diagnosis_12month', 'data_dir','img_dir']
tab_columns = [i for i in train.columns if i not in remove_list]

X_train = train.loc[:,tab_columns]
X_val = val.loc[:,tab_columns]
X_test = test.loc[:,tab_columns]

y_train = train.loc[:,'diagnosis_12month']
y_val = val.loc[:,'diagnosis_12month']
y_test = test.loc[:,'diagnosis_12month']

print((len(test[test['diagnosis_12month']==0])))
print((len(test[test['diagnosis_12month']==1])))
print((len(test[test['diagnosis_12month']==2])))


In [ ]:
pd.DataFrame(X_train.columns)

In [ ]:
all_X_train = pd.concat([X_train,X_val],axis=0)
all_y_train = pd.concat([y_train,y_val],axis=0)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# create a OneVsRestClassifier object with a logistic regression model
classifier = OneVsRestClassifier(LogisticRegression(C=0.01))

# perform 5-fold cross-validation and evaluate with the AUROC score
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

auroc_scores = cross_val_score(classifier, all_X_train, all_y_train,cv=cv, scoring="roc_auc_ovr")
f1_fivefold = cross_val_score(classifier, all_X_train, all_y_train, cv=cv, scoring="f1_macro")
acc_fivefold = cross_val_score(classifier, all_X_train, all_y_train, cv=cv, scoring="balanced_accuracy")

# classifier.fit(X_train, y_train)
# predictions = classifier.predict(X_test)

In [ ]:
acc_fivefold.mean(),auroc_scores.mean(),f1_fivefold.mean()

In [ ]:
classifier.fit(X_train, y_train)
predictions_probs = classifier.predict_proba(X_test)
predictions = np.argmax(predictions_probs,1)

In [ ]:
roc_auc_score(y_test,predictions_probs,multi_class="ovr")

In [ ]:
f1_score(y_test,predictions,average = 'macro')

In [ ]:
balanced_accuracy_score(y_test,predictions)

## XGboost

In [ ]:
from sklearn.model_selection import ShuffleSplit
from xgboost import XGBClassifier

# create an XGBClassifier with the "multi:softmax" objective and 3 classes
xgd_classifier = XGBClassifier(objective="multi:softmax", num_class=3, learning_rate=0.1, max_depth=15)


cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
auroc_fivefold = cross_val_score(xgd_classifier, all_X_train, all_y_train, cv=cv, scoring="roc_auc_ovr")
f1_fivefold = cross_val_score(xgd_classifier, all_X_train, all_y_train, cv=cv, scoring="f1_macro")
acc_fivefold = cross_val_score(xgd_classifier, all_X_train, all_y_train, cv=cv, scoring="balanced_accuracy")

print('acc:', acc_fivefold.mean())
print('auroc_ovr:', auroc_fivefold.mean())
print('f1_macro:', f1_fivefold.mean())

In [ ]:
xgd = xgd_classifier.fit(all_X_train, all_y_train)
xgd_classifier.feature_importances_

predictions_probs = xgd_classifier.predict_proba(X_test)
predictions = np.argmax(predictions_probs,1)

In [ ]:
roc_auc_score(y_test,predictions_probs,multi_class="ovr")

In [ ]:
f1_score(y_test,predictions,average = 'macro')

In [ ]:
balanced_accuracy_score(y_test,predictions)

In [ ]:
xgd_feature_importance = pd.DataFrame(list(xgd.get_booster().get_fscore().items()), \
                                      columns = ['feature','importance']).sort_values('importance', ascending=False)


from xgboost import plot_importance
plt.figure(figsize=(18,20))
plot_importance(xgd)
plt.show()

## Autoencoder

In [ ]:
import torch
from torch import nn, optim
from sklearn.metrics import roc_auc_score, f1_score, balanced_accuracy_score

class Autoencoder(nn.Module):
    """Makes the main denoising auto

    Parameters
    ----------
    in_shape [int] : input shape
    enc_shape [int] : desired encoded shape
    """

    def __init__(self, in_shape, out_cls, enc_shape = 8):
        super(Autoencoder, self).__init__()
        
        self.encode = nn.Sequential(
            nn.Linear(in_shape, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, enc_shape),
        )
        
        self.decode = nn.Sequential(
            nn.BatchNorm1d(enc_shape),
            nn.Linear(enc_shape, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, in_shape)
        )
        self.linear = nn.Linear(in_shape, out_cls)
        
    def forward(self, x):
        x = self.encode(x)
        x = self.decode(x)
        x = self.linear(x)
        return x

In [ ]:
#Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy(X_test, y_test, model):
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        output = model(X_test)
        y_pred = nn.functional.softmax(output, dim = 1)
        
        _, y_pred_class = y_pred.max(dim=1)
        #print(y_pred_class)#.shape,y_test.shape)
        auroc = roc_auc_score(y_test,y_pred,multi_class="ovr")
        f1 = f1_score(y_test,y_pred_class, average = 'macro')
        b_acc = balanced_accuracy_score(y_test,y_pred_class)
    return auroc, f1, b_acc

def train(model, error, optimizer, n_epochs, X, y, X_test, y_test):
    model.train()
    for epoch in range(1, n_epochs + 1):
        optimizer.zero_grad()
        output = model(X)
        loss = error(output, y)
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            
            auroc = testAccuracy(X_test, y_test, model)
            print(f'epoch {epoch} \t Test Loss: {loss.item():.4g} \t Test AUC: {auroc}')

In [ ]:
auto_encoder = Autoencoder(in_shape=X_train.shape[1], out_cls=3, enc_shape=2).double()

error = nn.CrossEntropyLoss()

optimizer = optim.Adam(auto_encoder.parameters(), lr = 0.001)


In [ ]:
X = torch.from_numpy(np.array(X_train))
y = torch.from_numpy(np.array(y_train))

X_t = torch.from_numpy(np.array(X_test))
y_t = torch.from_numpy(np.array(y_test))
train(auto_encoder, error, optimizer, 400,X,y, X_t, y_t )


In [ ]:
preds = np.load('/scratch/yx2105/shared/MLH/results/baseline_cnn_lr1_0/predictions_best_val.npy')

In [ ]:
auroc = roc_auc_score(y_val,preds,multi_class="ovr")

In [ ]:
auroc